## Load Packages

In [1]:
using Distributions

include("printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

# VaR for a N(μ,σ²) Return

$\textrm{VaR}_{95\%} = - 5^{th}$ percentile of the return distribution

With a $N(\mu,\sigma^2)$ distribution this gives

$\textrm{VaR}_{95\%} = - (\mu-1.64\sigma)$

In [3]:
function ϕNS(x,μ=0,σ²=1)                     #pdf of N(μ,σ²), NS for "non-standard"
    pdfx = pdf(Normal(μ,sqrt(σ²)),x)
    return pdfx
end

ϕNS (generic function with 3 methods)

In [4]:
μ = 8
σ = 16

R    = linspace(-60,60,301)
pdfR = ϕNS.(R,μ,σ^2)

q05   = μ-1.64*σ 
VaR95 = -(μ-1.64*σ)
printlnPs("with μ=$μ and \sigma=$σ, the 5th quantile and VaR 95% are: ",[q05 VaR95])

with μ=8 and sigma=16, the 5th quantile and VaR 95% are:    -18.240    18.240


In [6]:
plot(R,pdfR,color=:red,linewidth=2,legend=false)
title!("pdf of N($μ,$(σ^2))")
xlabel!("return, %")
plot!([-VaR95],linetype=:vline,color=:blue)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -60 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 pdf of N(8,256) 
 
 
 return, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 48.3815,369.674 50.2071,369.669 52.0327,369.664 53.8583,369.659 55.6839,369.652 57.5095,369.646 59.3351,369.638 61.1607,369.63 62.9863,369.621 64.8119,369.61 
 66.6375,369.599 68.4631,369.587 70.2887,369.574 72.1143,369.559 73.9399,369.543 75.7655,369.525 77.5911,369.506 79.4167,369.485 81.2423,369.462 83.0679,369.437 
 84.8936,369.409 86.7192,369.379 88.5448,369.347 90.3704,369.311 92.196,369.272 94.0216,369.23 95.8472,369.184 97.6728,369.134 99.4984,369.08 101.324,369.022 
 103.15,368.958 104.975,368.889 106.801,368.814 108.626,368.733 110.452,368.646 112.278,368.551 114.103,368.449 115.929,368.339 117.754,368.22 119.58,368.092 
 121.406,367.954 123.231,367.806 125.057,367.646 126.882,367.474 128.708,367.29 130.534,367.092 132.359,366.88 134.185,366.652 136.01,366.408 137.836,366.147 
 139.662,365.868 141.487,365.57 143.313,365.251 145.139,364.911 146.964,364.548 148.79,364.161 150.615,363.749 152.441,363.31 154.267,362.844 156.092,362.348 
 157.918,361.821 159.743,361.262 161.569,360.669 163.395,360.04 165.22,359.374 167.046,358.669 168.871,357.924 170.697,357.136 172.523,356.303 174.348,355.425 
 176.174,354.499 177.999,353.522 179.825,352.494 181.651,351.412 183.476,350.274 185.302,349.078 187.127,347.823 188.953,346.505 190.779,345.124 192.604,343.677 
 194.43,342.161 196.255,340.576 198.081,338.919 199.907,337.188 201.732,335.381 203.558,333.496 205.383,331.532 207.209,329.486 209.035,327.358 210.86,325.144 
 212.686,322.844 214.512,320.457 216.337,317.98 218.163,315.412 219.988,312.752 221.814,310 223.64,307.154 225.465,304.213 227.291,301.176 229.116,298.044 
 230.942,294.816 232.768,291.49 234.593,288.069 236.419,284.55 238.244,280.936 240.07,277.226 241.896,273.422 243.721,269.523 245.547,265.532 247.372,261.45 
 249.198,257.277 251.024,253.017 252.849,248.672 254.675,244.242 256.5,239.733 258.326,235.145 260.152,230.482 261.977,225.747 263.803,220.945 265.628,216.079 
 267.454,211.152 269.28,206.17 271.105,201.137 272.931,196.057 274.756,190.936 276.582,185.779 278.408,180.591 280.233,175.379 282.059,170.148 283.885,164.904 
 285.71,159.653 287.536,154.403 289.361,149.159 291.187,143.929 293.013,138.719 294.838,133.536 296.664,128.388 298.489,123.282 300.315,118.225 302.141,113.225 
 303.966,108.289 305.792,103.424 307.617,98.6383 309.443,93.9389 311.269,89.3333 313.094,84.8289 314.92,80.433 316.745,76.1528 318.571,71.9954 320.397,67.9678 
 322.222,64.0768 324.048,60.329 325.873,56.731 327.699,53.2889 329.525,50.0089 331.35,46.8967 333.176,43.9578 335.001,41.1976 336.827,38.621 338.653,36.2327 
 340.478,34.037 342.304,32.038 344.129,30.2393 345.955,28.6443 347.781,27.2559 349.606,26.0766 351.432,25.1088 353.258,24.3541 355.083,23.8141 356.909,23.4897 
 358.734,23.3815 360.56,23.4897 362.386,23.8141 364.211,24.3541 366.037,25.1088 367.862,26.0766 369.688,27.2559 371.514,28.6443 373.339,30.2393 375.165,32.038 
 376.99,34.037 378.816,36.2327 380.642,38.621 382.467,41.1976 384.293,43.9578 386.118,46.8967 387.944,50.0089 389.77,53.2889 391.595,56.731 393.421,60.329 
 395.246,64.0768 397.072,67.9678 398.898,71.9954 400.723,76.1528 402.549,80.433 404.374,84.8289 406.2,89.3333 408.026,93.9389 409.851,98.6383 411.677,103.424 
 413.502,108.289 415.328,113.225 417.154,118.225 418.979,123.282 420.805,128.388 422.631,133.536 424.456,138.719 426.282,143.929 428.107,149.159 429.933,154.403 
 431.759,159.653 433.584,164.904 435.41,170.148 437.235,175.379 439.061,180.591 440.887,185.779 442.712,190.936 444.538,196.057 446.363,201.137 448.189,206.17 
 450.015,211.152 451.84,216.079 453.666,220.945 455.491,225.747 457.317,230.482 459.143,235.145 460.96

# Loading Daily S&P 500 Data

In [7]:
xx = readdlm("Data/SP500RfPs.csv",',',header=true)
x  = xx[1]
SP = convert(Array{Float64},x[:,2])      #convert to numerical array, S&P 500 level
R  = (SP[2:end]./SP[1:end-1] - 1) * 100  #returns, % 
T  = length(R)  

println("Number of days in the sample: $T")

dN = Array{Date}(T)                      #convert to Date
for t = 1:T
    dN[t] = Date(x[t+1,1],"d/m/y")       #t+1 since the first return for this period 
end

Number of days in the sample: 9352


# Backtesting VaR from N() on Data

To backtest a VaR model, study the relative frequency of Loss > VaR. 

The code below does this for difference confidence levels (0.05,0.04,...) of the VaR.

In [8]:

μ_emp = mean(R)
σ_emp = std(R)

pval = collect(0.05:-0.005:0.005)
L    = length(pval)
Loss = -R

VaR       = fill(NaN,L)
VaR_emp   = fill(NaN,L)
CoverageN = fill(NaN,L)
for i = 1:L                 #loop over different (1-confidence levels)
    VaR_emp[i]   = -quantile(R,pval[i])
    critval      = abs(quantile(Normal(0,1),pval[i]))
    VaR[i]       = -(μ_emp - critval*σ_emp)
    CoverageN[i] = mean(Loss .> VaR[i])           #frequency of breaking the VaR
end    

println("conf level, empirical VaR, N()-based VAR, theoretical coverage, coverage")
printmat([1-pval VaR_emp VaR pval CoverageN])

conf level, empirical VaR, N()-based VAR, theoretical coverage, coverage
     0.950     1.640     1.790     0.050     0.041
     0.955     1.715     1.846     0.045     0.037
     0.960     1.802     1.907     0.040     0.034
     0.965     1.886     1.975     0.035     0.032
     0.970     2.031     2.052     0.030     0.029
     0.975     2.194     2.140     0.025     0.026
     0.980     2.349     2.244     0.020     0.023
     0.985     2.566     2.373     0.015     0.019
     0.990     2.958     2.547     0.010     0.016
     0.995     3.826     2.824     0.005     0.012



The code below studes the relative frequency of Loss > VaR, but this time over a sliding data window. This allows us to investigate if there are long periods of failure (in either direction) the VaR.

In [9]:
VaR95 = -(μ_emp - 1.64*σ_emp)

CoverageT = fill(NaN,T)
for t = 101:T
    CoverageT[t] = mean(Loss[t-100:t] .> VaR95)[1]
end    

#construct a good x-axis variable, eg. 2015.994624 for 30 Dec 2015
YearFrac = Dates.year(dN) + (Dates.month(dN)-1)/12 + (Dates.day(dN)-1)/(31*12)

plot(YearFrac,CoverageT*100,color=:blue,xlims=(1979,2016),ylims=(-1,35),legend=false)
title!("Pr(Loss > VaR 95%) over last 100 days")
ylabel!("%")
plot!([5],linetype=:hline,color=:black)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 Pr(Loss > VaR 95%) over last 100 days 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.0041,374.908 43.1665,374.908 43.2072,374.908 43.2478,374.908 43.2884,374.908 43.4103,374.908 43.4509,374.908 43.4915,374.908 43.5321,374.908 43.5727,374.908 
 43.6946,374.908 43.7352,374.908 43.7758,374.908 43.8164,374.908 43.8571,374.908 43.9789,374.908 44.0195,374.908 44.0601,374.908 44.1008,374.908 44.1414,374.908 
 44.2632,374.908 44.3039,374.908 44.3445,374.908 44.3851,374.908 44.4257,374.908 44.5882,374.908 44.6288,374.908 44.71,374.908 44.7507,374.908 44.8725,374.908 
 44.9131,374.908 44.9538,374.908 44.9944,374.908 45.035,374.908 45.1568,374.908 45.1975,374.908 45.2381,374.908 45.2787,374.908 45.3193,374.908 45.4412,374.908 
 45.4818,374.908 45.5224,374.908 45.563,374.908 45.6036,374.908 45.7255,374.908 45.7661,374.908 45.8067,374.908 45.8474,374.908 45.888,374.908 46.0098,374.908 
 46.0504,374.908 46.0911,374.908 46.1317,374.908 46.1723,374.908 46.2942,374.908 46.3348,374.908 46.3754,374.908 46.416,374.908 46.4566,374.908 46.5785,374.908 
 46.6191,374.908 46.6597,374.908 46.7003,374.908 46.741,374.908 46.8628,374.908 46.9034,374.908 46.944,374.908 46.9847,374.908 47.0253,374.908 47.1878,374.908 
 47.2284,374.908 47.269,374.908 47.3096,374.908 47.4315,374.908 47.4721,374.908 47.5127,374.908 47.5533,374.908 47.5939,374.908 47.7158,374.908 47.7564,374.908 
 47.797,374.908 47.8377,374.908 47.8783,374.908 48.0001,374.908 48.0407,374.908 48.0814,374.908 48.122,374.908 48.1626,374.908 48.3251,374.908 48.3657,374.908 
 48.4063,374.908 48.4469,374.908 48.4875,374.908 48.6094,374.908 48.65,364.964 48.6906,364.964 48.7313,364.964 48.7719,364.964 48.8937,364.964 48.9343,364.964 
 48.975,364.964 49.0156,364.964 49.0562,355.02 49.1781,355.02 49.2187,355.02 49.2593,355.02 49.2999,355.02 49.3405,355.02 49.4624,355.02 49.503,355.02 
 49.5436,355.02 49.5842,355.02 49.6249,355.02 49.7467,355.02 49.7873,355.02 49.8279,355.02 49.8686,355.02 49.9092,355.02 50.031,355.02 50.0717,355.02 
 50.1123,355.02 50.1529,355.02 50.1935,355.02 50.3154,355.02 50.356,355.02 50.3966,355.02 50.4778,355.02 50.5997,355.02 50.6403,355.02 50.6809,355.02 
 50.7216,355.02 50.7622,355.02 50.9246,355.02 50.9653,355.02 51.0059,355.02 51.0465,355.02 51.0871,355.02 51.209,355.02 51.2496,355.02 51.2902,355.02 
 51.3308,355.02 51.3714,355.02 51.4933,355.02 51.5339,355.02 51.5745,355.02 51.6152,355.02 51.6558,355.02 51.7776,355.02 51.8589,355.02 51.8995,355.02 
 51.9401,355.02 52.062,355.02 52.1432,345.076 52.1838,345.076 52.2244,345.076 52.3463,345.076 52.3869,345.076 52.4275,345.076 52.4681,345.076 52.5088,345.076 
 52.6306,345.076 52.6712,345.076 52.7118,345.076 52.7525,345.076 52.7931,345.076 52.9149,345.076 52.9556,345.076 52.9962,345.076 53.0368,345.076 53.0774,345.076 
 53.1993,345.076 53.2399,345.076 53.2805,345.076 53.3211,345.076 53.3617,345.076 53.4836,345.076 53.5242,345.076 53.5648,345.076 53.6055,345.076 53.6461,345.076 
 53.7679,345.076 53.8085,345.076 53.8492,345.076 53.8898,345.076 53.9304,345.076 54.0929,345.076 54.1335,345.076 54.1741,345.076 54.2147,345.076 54.3366,345.076 
 54.3772,345.076 54.4178,345.076 54.4584,345.076 54.4991,345.076 54.7021,345.076 54.7428,355.02 54.7834,355.02 54.824,345.076 54.8646,345.076 54.9865,345.076 
 55.0271,345.076 55.0677,345.076 55.1083,345.076 55.1489,355.02 55.2708,345.076 55.3114,345.076 55.352,345.076 55.3927,345.076 55.4333,345.076 55.5551,335.132 
 55.5957,335.132 55.6364,335.132 55.677,335.132 55.7176,335.132 55.8395,335.132 55.8801,335.132 55.9207,335.132 55.9613,335.132 56.1238,325.187 56.1644,325.187 
 56.205,325.187 56.2456,325.187 56.2863,325.187 56.4081,325.187 56.4487,325.187 56.4894,325.187 56.53,325.187 56.5706,325.187 56.6924,325.187 56.7331,325.1

# A Simple Dynamic VaR with Time-Varying Volatility

We first construct an simple estimate of $\sigma_t^2$ as a backward looking moving average

$\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1-\lambda) (R_{t-1} -\mu_{t-1})^2$,
where $\mu_{t-1}$ is the historical average return (including $t-2$)

Redo the VaR calculation using 

$\textrm{VaR}_{t} = - (\mu_t-1.64\sigma_t)$ and study if it has better properties than the static VaR

In [10]:
u = R - μ_emp                    #"residuals"

λ   = 0.9
vol = fill(σ_emp^2,T)
μ   = fill(μ_emp,T)
for t = 2:T
    μ[t]   = mean(R[1:t-1])   
    vol[t] = λ*vol[t-1] + (1-λ)*(R[t-1]-μ[t-1])^2    #RiskMetrics approach
end

CoverageN = fill(NaN,L)
for i = 1:L
    critval      = abs(quantile(Normal(0,1),pval[i]))
    VaR_i        = -(μ - critval*sqrt.(vol))
    CoverageN[i] = mean(Loss .> VaR_i)
end    

println("conf level, coverage")
printmat([1-pval CoverageN])

conf level, coverage
     0.950     0.061
     0.955     0.057
     0.960     0.053
     0.965     0.049
     0.970     0.045
     0.975     0.040
     0.980     0.034
     0.985     0.029
     0.990     0.022
     0.995     0.016



In [11]:
VaR95 = -(μ - 1.64*sqrt.(vol))

CoverageT = fill(NaN,T)
for t = 101:T
    CoverageT[t] = mean(Loss[t-100:t] .> VaR95[t-100:t])[1]
end    

YearFrac = Dates.year(dN) + (Dates.month(dN)-1)/12 + (Dates.day(dN)-1)/(31*12)

plot(YearFrac,CoverageT*100,color=:blue,xlims=(1979,2016),ylims=(-1,35),legend=false)
title!("Pr(Loss > VaR 95%) over last 100 days")
ylabel!("%")
plot!([5],linetype=:hline,color=:black)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 Pr(Loss > VaR 95%) over last 100 days 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.0041,305.299 43.1665,305.299 43.2072,295.355 43.2478,295.355 43.2884,295.355 43.4103,295.355 43.4509,295.355 43.4915,295.355 43.5321,295.355 43.5727,295.355 
 43.6946,305.299 43.7352,305.299 43.7758,305.299 43.8164,305.299 43.8571,305.299 43.9789,305.299 44.0195,305.299 44.0601,305.299 44.1008,305.299 44.1414,305.299 
 44.2632,305.299 44.3039,315.243 44.3445,315.243 44.3851,315.243 44.4257,325.187 44.5882,315.243 44.6288,315.243 44.71,315.243 44.7507,315.243 44.8725,315.243 
 44.9131,315.243 44.9538,315.243 44.9944,305.299 45.035,305.299 45.1568,305.299 45.1975,305.299 45.2381,305.299 45.2787,305.299 45.3193,305.299 45.4412,315.243 
 45.4818,315.243 45.5224,315.243 45.563,315.243 45.6036,315.243 45.7255,315.243 45.7661,315.243 45.8067,315.243 45.8474,315.243 45.888,315.243 46.0098,315.243 
 46.0504,315.243 46.0911,315.243 46.1317,315.243 46.1723,315.243 46.2942,315.243 46.3348,315.243 46.3754,315.243 46.416,315.243 46.4566,315.243 46.5785,315.243 
 46.6191,315.243 46.6597,315.243 46.7003,315.243 46.741,315.243 46.8628,315.243 46.9034,315.243 46.944,315.243 46.9847,315.243 47.0253,315.243 47.1878,305.299 
 47.2284,315.243 47.269,315.243 47.3096,315.243 47.4315,315.243 47.4721,315.243 47.5127,315.243 47.5533,315.243 47.5939,315.243 47.7158,315.243 47.7564,315.243 
 47.797,315.243 47.8377,315.243 47.8783,315.243 48.0001,315.243 48.0407,315.243 48.0814,315.243 48.122,325.187 48.1626,335.132 48.3251,335.132 48.3657,335.132 
 48.4063,335.132 48.4469,335.132 48.4875,335.132 48.6094,325.187 48.65,315.243 48.6906,315.243 48.7313,315.243 48.7719,315.243 48.8937,315.243 48.9343,315.243 
 48.975,315.243 49.0156,315.243 49.0562,305.299 49.1781,315.243 49.2187,315.243 49.2593,315.243 49.2999,315.243 49.3405,315.243 49.4624,315.243 49.503,315.243 
 49.5436,315.243 49.5842,315.243 49.6249,315.243 49.7467,315.243 49.7873,315.243 49.8279,315.243 49.8686,315.243 49.9092,315.243 50.031,315.243 50.0717,315.243 
 50.1123,315.243 50.1529,315.243 50.1935,315.243 50.3154,315.243 50.356,315.243 50.3966,315.243 50.4778,325.187 50.5997,325.187 50.6403,325.187 50.6809,325.187 
 50.7216,325.187 50.7622,325.187 50.9246,325.187 50.9653,335.132 51.0059,335.132 51.0465,335.132 51.0871,335.132 51.209,335.132 51.2496,335.132 51.2902,335.132 
 51.3308,335.132 51.3714,335.132 51.4933,335.132 51.5339,335.132 51.5745,335.132 51.6152,335.132 51.6558,335.132 51.7776,335.132 51.8589,335.132 51.8995,335.132 
 51.9401,335.132 52.062,335.132 52.1432,325.187 52.1838,325.187 52.2244,325.187 52.3463,325.187 52.3869,325.187 52.4275,325.187 52.4681,325.187 52.5088,325.187 
 52.6306,325.187 52.6712,325.187 52.7118,325.187 52.7525,325.187 52.7931,325.187 52.9149,325.187 52.9556,325.187 52.9962,325.187 53.0368,325.187 53.0774,325.187 
 53.1993,335.132 53.2399,335.132 53.2805,335.132 53.3211,335.132 53.3617,335.132 53.4836,335.132 53.5242,335.132 53.5648,335.132 53.6055,335.132 53.6461,335.132 
 53.7679,335.132 53.8085,335.132 53.8492,335.132 53.8898,325.187 53.9304,325.187 54.0929,325.187 54.1335,325.187 54.1741,325.187 54.2147,325.187 54.3366,325.187 
 54.3772,325.187 54.4178,325.187 54.4584,325.187 54.4991,325.187 54.7021,335.132 54.7428,345.076 54.7834,345.076 54.824,335.132 54.8646,335.132 54.9865,335.132 
 55.0271,335.132 55.0677,335.132 55.1083,335.132 55.1489,345.076 55.2708,335.132 55.3114,335.132 55.352,335.132 55.3927,335.132 55.4333,335.132 55.5551,325.187 
 55.5957,325.187 55.6364,325.187 55.677,325.187 55.7176,325.187 55.8395,325.187 55.8801,325.187 55.9207,325.187 55.9613,325.187 56.1238,325.187 56.1644,325.187 
 56.205,325.187 56.2456,325.187 56.2863,325.187 56.4081,325.187 56.4487,325.187 56.4894,325.187 56.53,

# Expected Shortfall

Recall: $\text{ES}_{\alpha}=-\text{E}(R|R\leq-\text{VaR}_{\alpha})$

For a normally distributed return $R\sim N(\mu,\sigma^{2})$ we have

$\text{ES}_{95\%}=-\mu+\frac{\phi(-1.64)}{0.05}\sigma$

In [12]:
μ = 8
σ = 16
ES95 = -(μ - ϕNS(1.64)/0.05*σ)
printlnPs("N()-based ES 95% with μ=$μ and \sigma=$σ is: ",ES95)

N()-based ES 95% with μ=8 and sigma=16 is:     25.268


In [13]:
ESN    = fill(NaN,L) 
ES_emp = fill(NaN,L)
for i = 1:L
    critval   = abs(quantile(Normal(0,1),pval[i]))    
    ESN[i]    = -(μ_emp - ϕNS(critval)/pval[i]*σ_emp)
    vv_i      = Loss .> VaR_emp[i]      
    ES_emp[i] = mean(Loss[vv_i])[1]        #mean of obs when Loss > VaR
end    

println("Conf level, expectec short fall from N(), Expected shortfall (historical)")
printmat([1-pval ESN ES_emp])

Conf level, expectec short fall from N(), Expected shortfall (historical)
     0.950     2.254     2.569
     0.955     2.303     2.668
     0.960     2.356     2.779
     0.965     2.415     2.914
     0.970     2.482     3.075
     0.975     2.560     3.270
     0.980     2.652     3.516
     0.985     2.767     3.871
     0.990     2.924     4.429
     0.995     3.176     5.628

